In [10]:
import pickle
from parameters.params import Params,Datasets,Models,Tasks
from util.env import set_param
from main import Main
import torch
from models.mine.MSTGAT import MSTGAT
from test_loop import test
from train_loop import train
import pandas as pd
from evaluate import IqrThreshold,MyConfusuion,IqrSensorThreshold,MinMaxThreshold,ZscoreThreshold

In [11]:
#board\my_mstgat_batadal\25_06_06_19_08_16
# snapshot\my_mstgat_batadal\25_06_06_21_32_18\0
path = "./snapshot/my_mstgat_batadal/25_06_07_07_11_55/0/"

param: Params = pickle.load(file=open(f"{path}param.pickle", "rb"))
param.val_ratio=0
set_param(param)
model_parameters=pickle.load(file=open( f"{path}model_parameters.pickle","rb"))
adj=torch.load(param.best_validationModel_path().replace("best.pt", "adj.pt"))

C:\Users\moji\AppData\Local\Temp\ipykernel_19552\2703351109.py:9: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  adj=torch.load(param.best_validationModel_path().replace("bes

In [ ]:
main = Main(param=param, modelParams=model_parameters, adj=adj)

# main.load_model(path=param.least_trainLossModel_path())


# DATASET 

*Datasets.batadal*
const:  attack  =  0.0
#####################################
sensors count:  27
actuators count:  8
consts count:  1
consts:  ['attack']
#####################################
#####################################
xlist (27):
['L_T2', 'L_T3', 'L_T4', 'L_T5', 'L_T6', 'L_T7', 'F_PU1', 'F_PU2', 'F_PU4', 'F_PU6', 'F_PU7', 'F_PU8', 'F_PU10', 'F_PU11', 'F_V2', 'P_J280', 'P_J269', 'P_J300', 'P_J256', 'P_J289', 'P_J415', 'P_J302', 'P_J306', 'P_J307', 'P_J317', 'P_J14', 'P_J422']
#####################################
Model is trained. Loading from file .....


RuntimeError: Error(s) in loading state_dict for MSTGAT:
	size mismatch for graph_learner.embedding.weight: copying a param with shape torch.Size([28, 32]) from checkpoint, the shape in current model is torch.Size([27, 32]).
	size mismatch for conv1d.weight: copying a param with shape torch.Size([28, 28, 16]) from checkpoint, the shape in current model is torch.Size([27, 27, 16]).
	size mismatch for conv1d.bias: copying a param with shape torch.Size([28]) from checkpoint, the shape in current model is torch.Size([27]).

In [ ]:
print(len(main.train_dataloader),len(main.val_dataloader),len(main.test_dataloader))

## Train losses
get all losses from train data loader

In [ ]:
train_all_loss,_,_ = test(main.model,main.train_dataloader,None)
train_all_loss.shape

## Test losses

In [ ]:
test_all_losses,ys,lbls = test(main.model, main.test_dataloader,confusion=None)


## IqrThreshold
From all losses get the threshold And confusion metrics.
Based onthe threshold and confusion, get the real output of test

In [ ]:
thr=IqrSensorThreshold()
thr.fit(train_all_loss)
conf=MyConfusuion(thr=thr).to(device=param.device)
print("test_all_losses.shape: ",test_all_losses.shape)
conf.update(test_all_losses,lbls)
confusion_matrix=conf.compute()
confusion_matrix

## MinMaxThreshold

In [ ]:

mmthr=MinMaxThreshold(multiplier=.95)
mmthr.fit(train_all_loss)
preds=mmthr.transform(test_all_losses)
mmConf=MyConfusuion(thr=mmthr).cuda()
mmConf.update(test_all_losses,lbls)
mmConf.compute()

In [ ]:
dft=pd.DataFrame({"FN":[],"FP":[],"m":[]})
for m in torch.arange(start=1.01,step=.0001,end=1.03):
    mxthr=MinMaxThreshold(multiplier=m)
    mxthr.fit(train_all_loss)
    mpreds=mxthr.transform(test_all_losses)
    mxConf=MyConfusuion(thr=mxthr).cuda()
    mxConf.update(test_all_losses,lbls)
    mtr=mxConf.compute().cpu().numpy()
    dft.loc[len(dft)]=[mtr[0][1],mtr[1][0],m.numpy()]
dft.plot(x="m")

In [ ]:
mmthr=ZscoreThreshold(threshold=3.42)
mmthr.fit(train_all_loss)
preds=mmthr.transform(test_all_losses)
mmConf=MyConfusuion(thr=mmthr).cuda()
mmConf.update(test_all_losses,lbls)
mmConf.compute()

In [ ]:
dft=pd.DataFrame({"FN":[],"FP":[],"m":[]})
for m in torch.arange(start=.8,step=.01,end=3.8):
    mxthr=ZscoreThreshold(threshold=m)
    mxthr.fit(train_all_loss)
    mpreds=mxthr.transform(test_all_losses)
    mxConf=MyConfusuion(thr=mxthr).cuda()
    mxConf.update(test_all_losses,lbls)
    mtr=mxConf.compute().cpu().numpy()
    dft.loc[len(dft)]=[mtr[0][1],mtr[1][0],m.numpy()]
dft.plot(x="m")

## AbsMaxThreshold

In [ ]:
from evaluate import AbsMaxThreshold
mmthr=AbsMaxThreshold(multiplier=3.7)
mmthr.fit(train_all_loss)
preds=mmthr.transform(test_all_losses)
mmConf=MyConfusuion(thr=mmthr).cuda()
mmConf.update(test_all_losses,lbls)
mmConf.compute()

In [ ]:

dft=pd.DataFrame({"FN":[],"FP":[]})
for m in torch.arange(start=.5,step=.1,end=5):
    mxthr=AbsMaxThreshold(multiplier=m)
    mxthr.fit(train_all_loss)
    mpreds=mxthr.transform(test_all_losses)
    mxConf=MyConfusuion(thr=mxthr).cuda()
    mxConf.update(test_all_losses,lbls)
    mtr=mxConf.compute().cpu().numpy()
    dft.loc[len(dft)]=[mtr[0][1],mtr[1][0]]
dft.plot()

In [ ]:
mmthr=IqrSensorThreshold()
# mmthr=MinMaxThreshold(multiplier=1.025)
mmthr.fit(train_all_loss)
preds=mmthr.transform(test_all_losses)

## PLOTTING


all test Datasets

In [ ]:

df=pd.DataFrame({"preds":preds.tolist(),"truth":lbls.tolist()})
df.plot()

Each attack zone


In [ ]:
import matplotlib.pyplot as plt
from util.data import get_attack_interval,getAttacks


attacks=getAttacks(df,"truth")
# Identify contiguous regions of ones

# Create subplots dynamically
num_regions = len(attacks)
fig, axes = plt.subplots(num_regions, 1, figsize=(8, 3*num_regions), sharex=False)

if num_regions == 1:
    axes = [axes]  # Ensure axes is iterable

for i in range(num_regions):
    # first_idx = group.index.min()  # First occurrence of 1
    # last_idx = group.index.max()   # Last occurrence of 1
    [first_idx,last_idx]=attacks[i]
    range_before = first_idx - int(2.0 *(last_idx-first_idx))
    range_after = last_idx + int(2.0 *(last_idx-first_idx))
    if range_before<0:
        range_before=0
    if range_after>len(df):
        range_after=last_idx
    # df.iloc[first_idx:last_idx].plot(ax=ax)
    axes[i].plot( df.loc[range_before:range_after,'truth'], label="Truth")
    axes[i].plot( df.loc[range_before:range_after,'preds'], color='red', label="Preds",linestyle="--")

    # ax.set_xlim(range_before, range_after)  # X-axis limit dynamically adjusted
    # ax.set(ybound=(0, max(df['preds']) + 10))  # Corrected method for Y-axis limits

    axes[i].set_title(f"Attack {i+1}")
    axes[i].legend()
    axes[i].grid(True)

plt.xlabel("Index")
plt.tight_layout()
plt.show()

In [ ]:
print("max: ",test_result[0].max(),
    #   val_result[0].max(),
      train_result[0].max())
print("min: ",test_result[0].min(),
    #   val_result[0].min(),
      train_result[0].min())


In [ ]:
from util.data import getAttacks

In [ ]:
minimum=0#val_result[0].min()
maximum=train_result[0].max().item()*1#*24.5

pred=torch.where((test_result[0]>maximum) ,torch.tensor(1),torch.tensor(0)).cpu().numpy()
labels_df=pd.DataFrame({"pred":pred,"truth":test_result[2].cpu().numpy()})
attacks=getAttacks(labels_df,"truth")


In [ ]:

for i,attack in enumerate(attacks):
    left=(attack[0]//100)*100
    right=((attack[1]//100)+1)*100
    labels_df.loc[left:right].plot(title=f"attack {i+1}")

In [ ]:
r=test_result[3].sum(-1)
print(r.max(),r.min(),r.mean())

In [ ]:
from evaluate import createMetrics
k=createMetrics(test_result,maximum)
print(k)
k.keys()

In [ ]:
k2=['TP', 'FP', 'TN', 'FN']
k1=["Accuracy","Precision","Recall","F1"]
d1={key:[] for key in k1}
d2={key:[] for key in k2}
x=[]
for i in range(100):
    _max=maximum*(.5+i/100)
    rr=createMetrics(test_result,_max)
    x.append(_max)
    for key  in d1.keys():
        d1[key].append(float(rr[key]))
    for key  in d2.keys():
        d2[key].append(float(rr[key]))